In [1]:
%matplotlib inline
import json
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
data = json.load(open('../data/items.jl'))

df = pd.io.json.json_normalize(data)

df = df.dropna(subset=['speech'])

df['speaker'] = df.apply(lambda x: x['speaker'][0].split('-'), axis=1)

df = df[df['speaker'].map(len) == 2]

df['state'] = df.apply(lambda x: x['speaker'][1], axis=1)

df['party'] = df.apply(lambda x: x['speaker'][0].split('%2C')[-1], axis=1)

df['speaker'] = df.apply(lambda x: x['speaker'][0].split('%2C')[0], axis=1)

df.party = pd.Categorical(df.party)

df['target'] = df.party.cat.codes

df['speech'] = df.apply(lambda x: ' '.join(x['speech']).replace('\t', ''), axis=1)

In [11]:
features_train, features_test, labels_train, labels_test = train_test_split(df['speech'], df['target'], test_size=0.1, random_state=42)

In [12]:
stopwords_pt = set(stopwords.words('portuguese') + list(punctuation))

In [138]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=stopwords.words('portuguese'))
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed = vectorizer.transform(features_test)
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train_transformed, labels_train)
features_train_transformed = selector.transform(features_train_transformed).toarray()
features_test_transformed = selector.transform(features_test_transformed).toarray()

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(C=10000, kernel='rbf')
clf.fit(features_train_transformed, labels_train)

pred = clf.predict(features_test_transformed)

print(accuracy_score(pred, labels_test))

In [23]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=68, random_state=4, min_samples_split=28)

clf.fit(features_train_transformed, labels_train)

pred = clf.predict(features_test_transformed)

print(accuracy_score(pred, labels_test))

0.8454789178649769


In [18]:
f = pd.Series(["Bandido bom é bandido morto"], index=[0])

In [19]:
ftest = vectorizer.transform(f)

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [20]:
ftest = selector.transform(ftest).toarray()

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [203]:
ftest.shape

(1, 5934)

In [204]:
p = clf.predict(ftest)

In [205]:
print(p)

[94]


In [206]:
# Problemão!
df['party'].value_counts()

PT                                                                                                                                                                                                        3056
PMDB                                                                                                                                                                                                      1206
PSOL                                                                                                                                                                                                      1104
PSB                                                                                                                                                                                                        871
PCDOB                                                                                                                                                                       

In [7]:
d = []

with open('../../data/sample.json', 'r') as f:
    for x in f:
        d.append(json.loads(x.replace('\n', '')))
        
df = pd.DataFrame(d)
df = df.dropna(subset=['discourse'])
df.party = pd.Categorical(df.party)
df['target'] = df.party.cat.codes

features_train, features_test, labels_train, labels_test = train_test_split(df['discourse'], df['target'], test_size=0.1, random_state=42)
stopwords_pt = set(stopwords.words('portuguese') + list(punctuation))

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=stopwords.words('portuguese'))
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed = vectorizer.transform(features_test)
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train_transformed, labels_train)
features_train_transformed = selector.transform(features_train_transformed).toarray()
features_test_transformed = selector.transform(features_test_transformed).toarray()

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [10]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=68, random_state=4, min_samples_split=28)

clf.fit(features_train_transformed, labels_train)

pred = clf.predict(features_test_transformed)

print(accuracy_score(pred, labels_test))

0.8454789178649769


In [9]:
from sklearn.metrics import accuracy_score

print(accuracy_score(pred, labels_test))

0.8454789178649769


In [21]:
p = clf.predict(ftest)

In [22]:
p

array([33], dtype=int8)

In [17]:
df['target'].value_counts()

 33    8944
 12    5379
 28    4102
 25    2168
 16    1902
 1     1867
 34    1842
 8     1761
 6     1661
 3     1580
 19    1467
 18    1403
 31    1083
 27     873
 10     744
 38     701
 26     637
-1      523
 20     465
 17     338
 40     304
 9      163
 22     157
 39     148
 30     120
 13      98
 42      89
 37      81
 35      78
 41      70
 36      68
 21      48
 15      45
 11      31
 7       22
 32      19
 0       12
 24       8
 23       7
 29       7
 14       4
 2        3
 5        1
 4        1
Name: target, dtype: int64